In [ ]:
import pandas as pd
import os
print("Current directory:", os.getcwd())
import sys
sys.path.append('.')
from MDQN import M_DQN_Agent
import torch
from collections import deque
from Trading_Simulator import TradingEnvironment, pca_res_gen, fourier_signal_extractor

%reload_ext autoreload
%autoreload 2

In [ ]:
env = TradingEnvironment(financial_dataset=price_df,
                        residual_generator=pca_res_gen,
                        signal_generator=fourier_signal_extractor,
                        episode_length=100,
                        lookback_window=252,
                        signal_window=60,
                        transaction_costs=0.0,
                        short_cost=0.0)

In [ ]:
import os
filename = 'price_df.csv'
if os.path.exists(filename):
    price_df = pd.read_csv(filename)
    price_df = price_df.rename(columns={'Unnamed: 0':'date'})
    price_df.set_index('date', inplace=True)
    price_df.index = pd.to_datetime(price_df.index)

returns = price_df.diff(1)

In [ ]:

# PARAMETERS
#seed = 100
seed = np.random.randint(0,100000)

#writer = SummaryWriter("runs/"+"DQN_LL_new_1")
frames = 1000
BUFFER_SIZE = 1000000
BATCH_SIZE = 64
GAMMA = 0.99
TAU = 1e-2
eps_frames=5000
min_eps=0.025
LR = 1e-3
UPDATE_EVERY = 1
n_step = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using ", device)
eps_fixed = False
SAVE_MODEL = False
file_name = 'N1'

# Set seeds
#env.seed(seed)
#nv.action_space.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

# action_size     = env.action_space.n #### going to fix this
action_size       = 3
#state_size        = env.observation_space.shape
state_size        = [16]

agent = M_DQN_Agent(state_size=state_size,    
                        action_size=action_size,
                        layer_size=64,
                        BATCH_SIZE=BATCH_SIZE, 
                        BUFFER_SIZE=BUFFER_SIZE, 
                        LR=LR, 
                        TAU=TAU, 
                        GAMMA=GAMMA, 
                        UPDATE_EVERY=UPDATE_EVERY, 
                        device=device, 
                        seed=seed)

##########################
action_to_portfolio = {0:-1, 1:0, 2: 1}
scores = []                        # list containing scores from each episode
scores_window = deque(maxlen=100)  # last 100 scores
output_history = []
frame = 0
if eps_fixed:
    eps = 0
else:
    eps = 1
eps_start = 1
i_episode = 1

env = TradingEnvironment(financial_dataset=price_df.dropna(axis=0, thresh=300),
                        residual_generator=pca_res_gen,
                        signal_generator=fourier_signal_extractor,
                        episode_length=100,
                        lookback_window=252,
                        signal_window=60,
                        transaction_costs=0.0,
                        short_cost=0.0)

state, _ = env.warm_up()
score = 0

for frame in range(1, frames+1):
    action = agent.act_para(state, eps)
    
    next_state, reward, done, _ = env.step([action_to_portfolio[action]])
    agent.step(state, action, reward, next_state, done)#, writer)
    state = next_state
    score += reward
    # linear annealing to the min epsilon value until eps_frames and from there slowly decease epsilon to 0 until the end of training
    if eps_fixed == False:
        if frame < eps_frames:
            eps = max(eps_start - (frame*(1/eps_frames)), min_eps)
        else:
            eps = max(min_eps - min_eps*((frame-eps_frames)/(frames-eps_frames)), 0.001)
    
    if done:
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        #writer.add_scalar("Average100", np.mean(scores_window), frame)
        output_history.append(np.mean(scores_window))
        env.evaluate_performance()
        print('\rEpisode {}\tFrame {} \tAverage Score: {:.2f}'.format(i_episode, frame, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tFrame {}\tAverage Score: {:.2f}'.format(i_episode,frame, np.mean(scores_window)))
        i_episode +=1 
        score = 0  